# Generate QnA synthetic dataset from CSV


In [1]:
from dotenv import load_dotenv
import os, shutil, random
from langchain_community.document_loaders.csv_loader import CSVLoader
from util.preprocess import convert_html_to_md
from util.common_utils import get_language_code

load_dotenv()

raw_data_dir = "../raw_data"
file_path = f"{raw_data_dir}/csv/en-store-information-virtual.csv"

DOMAIN = "CS (Customer Support)"
LANGUAGE = "English" # You can change your language here. e.g., "Korean", "Japanese", "Chinese"
LANGUAGE_CODE = get_language_code(LANGUAGE)
print(f"Domain: {DOMAIN}, Language: {LANGUAGE}, Language Code: {LANGUAGE_CODE}")

Domain: CS (Customer Support), Language: English, Language Code: en



## 1. Read & Preprocess CSV file
---

In [2]:
%%time

loader = CSVLoader(
    file_path=file_path,
    csv_args={
        "delimiter": ",",
        "quotechar": '"',
    },    
)
docs = loader.load()

CPU times: user 1.09 ms, sys: 2.06 ms, total: 3.16 ms
Wall time: 4.14 ms


In [3]:
preprocessed_docs = []
for doc in docs:
    md = convert_html_to_md(doc.page_content)
    preprocessed_docs.append(md)

print(preprocessed_docs[0])

No.: 1
Store Name: BrightStore Central
Address: 3rd Floor, Star Tower, 10000 Myeongdong-ro, Jung-gu, Seoul
Zip Code: 04567
Phone Number: 02-5678-1234
Fax Number: 02-4321-8765
Store Size: 150 pyeong
Closed: N
Consultant Available: Y
Service Center Available: Y
Store Description: BrightStore Central offers a wide range of the latest electronics and friendly consultation services. We promise to provide the best service for your convenience.


## 2. Construct QnA Pairs
---

In [4]:
from util.qa import CustomQADataGenerator
model_config = {
    "deployment": os.getenv("AZURE_OPENAI_DEPLOYMENT_NAME"),
    "model": "gpt-4o",
    "max_tokens": 2000,
}

qa_generator = CustomQADataGenerator(model_config=model_config, templates_dir=f"./prompt_template/{LANGUAGE_CODE}")

In [5]:
import asyncio
from collections import Counter
from typing import Dict
import os
from azure.ai.generative.synthetic.qa import QAType
concurrency = 6  # number of concurrent calls
sem = asyncio.Semaphore(concurrency)

#qa_type = QAType.CONVERSATION
qa_type = QAType.LONG_ANSWER

async def generate_async(text: str) -> Dict:
    async with sem:
        return await qa_generator.generate_async(
            text=text,
            qa_type=qa_type,
            num_questions=3,  # Number of questions to generate per text
        )

In [8]:
preprocessed_docs[:2]

['No.: 1\nStore Name: BrightStore Central\nAddress: 3rd Floor, Star Tower, 10000 Myeongdong-ro, Jung-gu, Seoul\nZip Code: 04567\nPhone Number: 02-5678-1234\nFax Number: 02-4321-8765\nStore Size: 150 pyeong\nClosed: N\nConsultant Available: Y\nService Center Available: Y\nStore Description: BrightStore Central offers a wide range of the latest electronics and friendly consultation services. We promise to provide the best service for your convenience.',
 'No.: 2\nStore Name: TechPlanet Seocho\nAddress: 4th Floor, Hi Tower, 11111 Seocho-daero, Seocho-gu, Seoul\nZip Code: 06654\nPhone Number: 02-6789-2345\nFax Number: 02-5432-9876\nStore Size: 180 pyeong\nClosed: N\nConsultant Available: N\nService Center Available: Y\nStore Description: At TechPlanet Seocho, you can experience a variety of electronic products firsthand and receive personalized services from our expert consultants.']

In [7]:
input_batch = preprocessed_docs
results = await asyncio.gather(*[generate_async(text) for text in input_batch], return_exceptions=True)

question_answer_list = []
token_usage = Counter()
for result in results:
    if isinstance(result, Exception):
        raise result  # exception raised inside generate_async()
    question_answer_list.append(result["question_answers"])
    token_usage += result["token_usage"]

print("Successfully generated QAs")

Successfully generated QAs


## 3. Save to jsonl for fine-tuning
---

In [9]:
import json
from util.common_utils import convert_to_oai_format, save_jsonl

output_dir = './dataset'
os.makedirs(output_dir, exist_ok=True)

system_prompt_msg = f"""You are the SME (Subject Matter Expert) in {DOMAIN}. Please answer the questions accurately. If the question is in {LANGUAGE}, write your answer in {LANGUAGE}."""

save_filename = "store-info"
oai_qa_pair = convert_to_oai_format(question_answer_list, system_prompt_msg=system_prompt_msg)

#save_jsonl(qa_pair, f"{output_dir}/{save_filename}.jsonl")
save_jsonl(oai_qa_pair, f"{output_dir}/{save_filename}-oai.jsonl")